In [2]:
import ydbf
import pandas as pd
import os

In [3]:
BANKS_LIST = [2306, 2879, 415, 1455, 1973, 2776,
              3231, 2880, 2860, 1810, 3303, 3475, 2590, 2587, 567, 53, 2119, 2602, 2659,
              2070, 1326, 3460, 2914, 1281, 2990, 444, 3176, 436, 3224, 3538, 3505, 2312, 3138, 3255,
              2216, 708, 2684, 2494, 1659, 3269, 2506, 2574, 3415, 2816, 2799, 3527, 2929, 760, 435, 2883, 101,
              2368, 2748, 3482, 3407, 1144, 1745, 2309, 841, 1291, 598, 524, 2, 356, 903, 3173, 1896, 1460, 3287,
              1000, 1557, 2796, 3223, 354, 665, 3252, 2576, 2490, 1046, 2438, 3490, 2982, 256, 1948, 843, 646, 3330, 2738,
              2629, 3061, 3328, 857, 492, 1818, 990, 1765, 1781, 2402, 3161, 2645, 1809, 2065, 918, 817, 574, 2900, 2559, 1732,
              3519, 1829, 2763, 2495, 600, 3175, 2609, 2390, 2867, 2802, 1151, 438, 575, 1189, 842, 2241, 3298, 65, 3333, 3446, 1087,
              3184, 2245, 1115, 3300, 1680, 3311, 2584, 2905, 2682, 2499, 2607, 2518, 478, 2868, 1158, 609, 2568, 3525, 1920, 1343, 2707,
              3379, 3395, 2407, 2618, 3473, 2440, 2443, 3337, 3396, 1927, 77, 3247, 912, 3172, 1978, 1751, 3365, 2789, 2722, 3340, 1052, 2268,
              2103, 695, 2374, 3390, 1949, 3077, 2170, 3421, 1966, 2876, 702, 2755, 2546, 1352, 1747, 3417, 2932, 3202, 2650, 902, 3124, 3437,
              1016, 2611, 3516, 2766, 965, 3271, 604, 3436, 2110, 875, 2551, 2534, 650, 1788, 21, 3001, 2733, 537, 212, 2996, 3251, 538, 2638, 3296,
              3013, 3292, 3413, 1067, 3354, 3095, 3450, 3017, 170, 2272, 3073, 1573, 3349, 3099, 1354, 3257, 328, 481, 2790, 783, 3403, 779, 685, 2289,
              3293, 85, 3468, 330, 2048, 1481, 1637, 3368, 2721, 3507, 2152, 106, 2034, 2168, 3245, 2015, 2838, 2846, 3194, 2557, 705, 2664, 804, 1376, 963, 2830,
              1329, 554, 2881, 2307, 236, 1288, 2853, 2995, 3494, 588, 3235, 2304, 3136, 2085, 1312, 480, 2252, 1581, 2673, 3470, 2507, 1720, 933, 2210, 2807, 3279,
              2997, 3416, 2964, 2275, 1370, 429, 249, 3479, 3388, 2209, 3500, 1885, 2208, 1143, 1049, 1114, 316, 254, 2225, 121, 3531, 3515, 3529, 485, 493, 3290, 3116,
              1319, 2530, 2998, 1399, 3465, 67, 1307, 3185, 3463, 2772, 1, 3467, 3027, 2564
              ]

ACCOUNTS_LIST_101 = ['60401', '60404', '60405',
                     '60414', '60415', '61901', '61902',
                     '61903', '61904', '61905', '61906', '61907',
                     '61908', '61909', '61910', '61911', '61912',
                     '62001', '62002', '62101', '62102', '62103']

ACCOUNTS_LIST_135 = ['8637.0', '8695.0', '8826.0',
                     '8828.0', '8829.0', '8835.0', '8880']


In [11]:
def parse_101_dbf(filename):
    fh = open(filename, 'rb')
    dbf_101 = ydbf.open(fh, encoding='cp866')

    df_list_101 = []

    for i, record in enumerate(dbf_101):
        bank_id = record['REGN']
        account_number = record['NUM_SC']
        if bank_id in BANKS_LIST and account_number in ACCOUNTS_LIST_101:
            new_row = {'bank_id': bank_id,
                    'ishod_ostatki': record['VITG'], # Входящие остатки «итого», тыс. руб.
                    'vhod_ostatki': record['IITG'], # Исходящие остатки «итого», тыс. руб.
                    'account': account_number,
                    'date': record['DT']
                    }
            new_row_df = pd.DataFrame(data=new_row, index=[1])
            df_list_101.append(new_row_df)
        else:
            pass 

    full_df_101 = pd.concat(df_list_101)
    
    return full_df_101

def parse_135_dbf(filename, date):
    fh = open(filename, 'rb')
    dbf_135 = ydbf.open(fh, encoding='cp866')

    df_list_135 = []

    for i, record in enumerate(dbf_135):
        bank_id = record['REGN']
        account_number = record['C1_1']
        if bank_id in BANKS_LIST and account_number in ACCOUNTS_LIST_135:
            new_row = {'bank_id': bank_id,
                    'amount': record['C2_1'],
                    'account': account_number,
                    }
            new_row_df = pd.DataFrame(data=new_row, index=[1])
            df_list_135.append(new_row_df)
        else:
            pass

    full_df_135 = pd.concat(df_list_135)
    full_df_135['date'] = date
    full_df_135['account'] = full_df_135['account'].astype(float).astype(int)

    return full_df_135


def parse_123_dbf(filename, date):
    fh = open(filename, 'rb')
    dbf_123 = ydbf.open(fh, encoding='cp866')

    df_list_123 = []

    for i, record in enumerate(dbf_123):
        bank_id = record['REGN']
        indicator_id = record['C1']
        if bank_id in BANKS_LIST and indicator_id == "000":
            new_row = {'bank_id': bank_id,
                    'amount': record['C3']
                    # 'account': account_number,
                    }
            new_row_df = pd.DataFrame(data=new_row, index=[1])
            df_list_123.append(new_row_df)
        else:
            pass

    full_df_123 = pd.concat(df_list_123)
    full_df_123['date'] = date

    return full_df_123


def parse_135_part3_dbf(filename):
    fh = open(filename, 'rb')
    dbf = ydbf.open(fh, encoding='cp866')

    df_list = []

    for i, record in enumerate(dbf):
        bank_id = record['REGN']
        ratio_name = record['C1_3']
        ratio_value = record['C2_3']
        ratio_ref_value = record['C3_3']
        if bank_id in BANKS_LIST and ratio_name == 'Н1.0':
            new_row = {'bank_id': bank_id,
                    'ratio_name': ratio_name,
                    'ratio_value': ratio_value,
                    'ratio_ref_value': ratio_ref_value
                    }
            new_row_df = pd.DataFrame(data=new_row, index=[1])
            df_list.append(new_row_df)
        else:
            pass

    full_df = pd.concat(df_list)

    return full_df


In [12]:
full_df_135_pt3 = parse_135_part3_dbf('135-20210701/062021_135_3.dbf')

In [14]:
full_df_135_pt3.to_clipboard()

# Форма 101

In [5]:
full_df_101_022021 = parse_101_dbf('022021B1.DBF')
full_df_101_062021 = parse_101_dbf('062021B1.DBF')

In [6]:
full_df_101 = pd.merge(left=full_df_101_022021, right=full_df_101_062021, left_on=[
                       'bank_id', 'account'], right_on=['bank_id', 'account'], how='outer', suffixes=('_march21', '_jul21'))

# Форма 135

In [8]:
full_df_135_022021 = parse_135_dbf('022021_135_1.dbf', date='2021-03-01')
full_df_135_062021 = parse_135_dbf('062021_135_1.dbf', date='2021-07-01')
full_df_135 = pd.merge(left=full_df_135_022021, right=full_df_135_062021, left_on=[
                       'bank_id', 'account'], right_on=['bank_id', 'account'], how='outer', suffixes=('_march21', '_jul21'))


# Форма 123

In [22]:
full_df_123_062021 = parse_123_dbf('062021_123D.dbf', date='2021-07-01')

# Match original Excel formatting and include all necessary accounts

In [9]:
def cross_join_banks_accounts(banks_list, accounts_list):
    """Make a cross join df to include all banks and all accounts"""
    banks_df = pd.DataFrame({'bank_id': banks_list})
    all_accounts_df = pd.DataFrame({'account': accounts_list})
    all_banks_accounts = pd.merge(left=banks_df, right=all_accounts_df, how='cross')
    return all_banks_accounts




all_banks_accounts_101 = cross_join_banks_accounts(BANKS_LIST, ACCOUNTS_LIST_101)

all_banks_accounts_135 = cross_join_banks_accounts(BANKS_LIST, ACCOUNTS_LIST_135)

all_banks_accounts_135['account'] = all_banks_accounts_135['account'].astype(float).astype(int)

In [26]:
all_banks_accounts_101_merged = pd.merge(left=all_banks_accounts_101, right=full_df_101, left_on=['bank_id', 'account'], right_on=['bank_id', 'account'], how='left')
all_banks_accounts_135_merged = pd.merge(left=all_banks_accounts_135, right=full_df_135, left_on=['bank_id', 'account'], right_on=['bank_id', 'account'], how='left')
all_banks_accounts_123_merged = pd.merge(left=pd.DataFrame({'bank_id': BANKS_LIST}), right=full_df_123_062021, left_on=['bank_id'], right_on=['bank_id'], how='left')

In [11]:
# pivot rows to columns for 135 accounts
pivoted_135 = all_banks_accounts_135_merged.pivot(index='bank_id', columns='account', values=['amount_march21', 'amount_jul21']).fillna(0)

In [27]:
# fill missed values with 0

all_banks_accounts_101_final = all_banks_accounts_101_merged.drop(['date_march21', 'date_jul21'], axis=1).copy()
all_banks_accounts_101_final = all_banks_accounts_101_final.fillna(0)
all_banks_accounts_123_final = all_banks_accounts_123_merged

In [31]:
# Cannot overwrite existing sheets, you need to delete old sheets before executing command
with pd.ExcelWriter('ecosystems_calculations_capital.xlsx', engine='openpyxl',
                    mode='a') as writer:
    all_banks_accounts_101_final.to_excel(writer, sheet_name='101_parsed', index=False)
    pivoted_135.to_excel(writer, sheet_name='135_parsed')
    all_banks_accounts_123_final.to_excel(writer, sheet_name='123_parsed', index=False)